In [1]:
import numpy as np
import pandas as pd

In [2]:
df_train = pd.read_csv('quora-question-pairs/train.csv')

In [3]:
df_train.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [4]:
df_train[df_train['is_duplicate']==1].head()

,id,qid1,qid2,question1,question2,is_duplicate
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
11,11,23,24,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,1
12,12,25,26,What can make Physics easy to learn?,How can you make physics easy to learn?,1
13,13,27,28,What was your first sexual experience like?,What was your first sexual experience?,1


In [5]:
df_train.shape

(404290, 6)

In [6]:
df_train = df_train.sample(30000, random_state=42)

In [7]:
df_train.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [8]:
is_NaN = df_train.isnull()
is_NaN

,id,qid1,qid2,question1,question2,is_duplicate
8067,False,False,False,False,False,False
368101,False,False,False,False,False,False
70497,False,False,False,False,False,False
226567,False,False,False,False,False,False
73186,False,False,False,False,False,False
...,...,...,...,...,...,...
81275,False,False,False,False,False,False
273382,False,False,False,False,False,False
142371,False,False,False,False,False,False
79120,False,False,False,False,False,False


In [9]:
row_has_NaN = is_NaN.any(axis=1)
row_has_NaN

8067      False
368101    False
70497     False
226567    False
73186     False
          ...  
81275     False
273382    False
142371    False
79120     False
213244    False
Length: 30000, dtype: bool

In [10]:
rows_with_NaN = df_train[row_has_NaN]
rows_with_NaN

,id,qid1,qid2,question1,question2,is_duplicate


In [11]:
df_train.dropna(inplace=True)
df_train.shape

(30000, 6)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=3000)

In [13]:
questions = list(df_train['question1']) + list(df_train['question2'])
questions[:5]

['How do I play Pokémon GO in Korea?',
 'What are some of the best side dishes for crab cakes?',
 'Which is more advisable and better material for a crash test in automobiles, ductile or brittle?',
 'How do I improve logical programming skills?',
 'How close we are to see 3rd world war?']

In [14]:
q1_arr, q2_arr = np.vsplit(tfidf.fit_transform(questions).toarray(), 2)

In [15]:
df1 = pd.DataFrame(q1_arr, index=df_train.index)
df2 = pd.DataFrame(q2_arr, index=df_train.index)
df = pd.concat([df1, df2], axis=1)
df.shape

(30000, 6000)

In [16]:
df['is_duplicate'] = df_train['is_duplicate']

In [17]:
X = df.iloc[:, 0:-1].values
y = df.iloc[:, -1].values

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test, y_pred)

0.7438333333333333

In [20]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test,y_pred)

/Users/yashjain/Downloads/Development/tensorflow-env/env/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/yashjain/Downloads/Development/tensorflow-env/env/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:46:23] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117899018/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.7335